janky code for the NumpyLoader dataset class.

In [42]:
import json
import os
from functools import partial
import numpy as np
from datasets.utils import walk_files
from sklearn.preprocessing import normalize, MultiLabelBinarizer
import joblib
from torch.utils.data import DataLoader
from datasets import CinC2020Beats, NumpyLoader
import matplotlib.pyplot as plt

In [51]:
with open("data/snomed_ct_dx_map.json", "r") as f:
    code_mapping = json.load(f)
label_keys = sorted(list(map(int, code_mapping.keys())))
mlb = MultiLabelBinarizer(classes = label_keys)
mlb.fit(label_keys)
# mlb.transform([[label_keys[0],]])

MultiLabelBinarizer(classes=[368009, 6374002, 10370003, 11157007, 13640000,
                             17338001, 27885002, 28189009, 29320008, 39732003,
                             47665007, 49260003, 49578007, 50799005, 53741008,
                             54016002, 54329005, 55930002, 57054005, 59118001,
                             59931005, 60423000, 61277005, 63593006, 65778007,
                             67198005, 74390002, 74615001, 75532003, 77867006, ...])

In [11]:
ds = CinC2020Beats()
print(len(ds))
def load_stackable_cache(ds_out):
    valid_windows, age, sex, dx = ds_out

    num_windows, _ = valid_windows.shape
    
    age = np.repeat(age, num_windows).astype(np.intc)
    sex = np.repeat(sex, num_windows).astype(np.intc)
    dx = np.repeat(dx, num_windows, axis=0).astype(np.bool_)
    return valid_windows, age, sex, dx

print([v.shape for v in load_stackable_cache(ds[0])])
# ds[i] for i in range(len(ds))

767097
[(400, 12), (400,), (400,), (400, 123)]


In [61]:
def load_stackable_cache(npz_file):
    with open(npz_file, "rb") as f:
        valid_windows = np.load(f)
        age = np.load(f)
        sex = np.load(f)
        dx = np.load(f)

    valid_windows = valid_windows.astype(np.float32)
    np.nan_to_num(valid_windows, copy=False)
    valid_windows = np.stack([normalize(w.T).T for w in valid_windows])
    num_windows, _, _ = valid_windows.shape
    age = np.repeat(age, num_windows).astype(np.intc)
    sex = np.repeat(sex, num_windows).astype(np.intc)
    
    sparse_dx = mlb.transform([dx,])
    sparse_dx = np.repeat(sparse_dx, num_windows, axis=0).astype(np.bool_)

    return valid_windows, age, sex, sparse_dx

npz_files = tuple(walk_files(
    "data/.cache_beat_400", suffix=".npz", prefix=True, remove_suffix=False
))
# zip(*map(load_stackable_cache, npz_files))
valid_windows, age, sex, sparse_dx = load_stackable_cache(npz_files[0])
# plt.plot(valid_windows[0])
# plt.show()
# plt.plot(ds[0][0])
# plt.show()
# sparse_dx == ds[0][3]

print(valid_windows.shape, age.shape, sex.shape, sparse_dx.shape)

(12, 400, 12) (12,) (12,) (12, 123)


In [64]:

# valid_windows, age, sex, dx = zip(*map(load_stackable_cache, [ds[i] for i in range(10)]))
valid_windows, age, sex, dx = zip(*map(load_stackable_cache, npz_files))

# valid_windows, age, sex, dx = zip(*joblib.Parallel(n_jobs=-1, verbose=1)(
#     joblib.delayed(load_stackable_cache)(npz_file)
#     for npz_file in npz_files[:2]
# ))
if not os.path.isfile("cache_beat_400_windows.npy"):
    with open("cache_beat_400_windows.npy", "wb") as f:
        _windows = np.concatenate(valid_windows)
        print("windows", _windows.shape)
        np.save(f, _windows)
if not os.path.isfile("cache_beat_400_age.npy"):
    with open("cache_beat_400_age.npy", "wb") as f:
        _age = np.concatenate(age)
        print("age", _age.shape)
        np.save(f, _age)
if not os.path.isfile("cache_beat_400_sex.npy"):
    with open("cache_beat_400_sex.npy", "wb") as f:
        _sex = np.concatenate(sex)
        print("sex", _sex.shape)
        np.save(f, _sex)
if not os.path.isfile("cache_beat_400_dx.npy"):
    with open("cache_beat_400_dx.npy", "wb") as f:
        _dx = np.concatenate(dx)
        print("dx", _dx.shape)
        np.save(f, _dx)

windows (767097, 400, 12)
age (767097,)
sex (767097,)
dx (767097, 123)


In [71]:
dx[0]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [39]:
c_ds = NumpyLoader("cache_beat_400_windows.npy")
c_ds[0].shape

(400, 12)

In [36]:
with open("data/snomed_ct_dx_map.json", "r") as f:
    key_map = json.load(f)
key_map

{'270492004': ['IAVB', '1st degree av block'],
 '195042002': ['IIAVB', '2nd degree av block'],
 '61277005': ['AIVR', 'accelerated idioventricular rhythm'],
 '426664006': ['AJR', 'accelerated junctional rhythm'],
 '413444003': ['AMIs', 'acute myocardial ischemia'],
 '426434006': ['AnMIs', 'anterior ischemia'],
 '54329005': ['AnMI', 'anterior myocardial infarction'],
 '164889003': ['AF', 'atrial fibrillation'],
 '195080001': ['AFAFL', 'atrial fibrillation and flutter'],
 '164890007': ['AFL', 'atrial flutter'],
 '195126007': ['AH', 'atrial hypertrophy'],
 '251268003': ['AP', 'atrial pacing pattern'],
 '713422000': ['ATach', 'atrial tachycardia'],
 '233917008': ['AVB', 'av block'],
 '251170000': ['BPAC', 'blocked premature atrial contraction'],
 '74615001': ['BTS', 'brady tachy syndrome'],
 '426627000': ['Brady', 'bradycardia'],
 '418818005': ['Brug', 'brugada syndrome'],
 '6374002': ['BBB', 'bundle branch block'],
 '426749004': ['CAF', 'chronic atrial fibrillation'],
 '413844008': ['CMIs'